**To Do:**
1. clean the collected urls to only include lates version ( removes duplication)

In [43]:
import requests
import pandas as pd

start_date = '2023-01'
end_date = '2023-12'




url = "https://developer.uspto.gov/products/bdss/get/ajax"
params = {
    "data": f'{{"name":"APPXML","fromDate":"{start_date}","toDate":"{end_date}"}}'
}
response = requests.get(url, params=params)
if response.status_code != 200:
    print(f"Error: {response.status_code}")
else:
    print("Urls collected for the period from {} to {} successfully".format(start_date, end_date)) 



In [47]:
response.url

'https://developer.uspto.gov/products/bdss/get/ajax?data=%7B%22name%22%3A%22APPXML%22%2C%22fromDate%22%3A%222023-01%22%2C%22toDate%22%3A%222023-12%22%7D'

In [64]:
data = response.json()
rows = []
for f in data["productFiles"]:
    rows.append([f["fileFromTime"],f["fileName"], f["fileSize"]/1000000, f["fileDownloadUrl"]])

df = pd.DataFrame(rows, columns=["Date","File Name", "File Size (Mb)", "Download URL"])
df

,Date,File Name,File Size (Mb),Download URL
0,2023-01-05,ipa230105.zip,168.474121,https://bulkdata.uspto.gov/data/patent/applica...
1,2023-01-12,ipa230112_r1.zip,85.222230,https://bulkdata.uspto.gov/data/patent/applica...
2,2023-01-12,ipa230112.zip,85.221947,https://bulkdata.uspto.gov/data/patent/applica...
3,2023-01-19,ipa230119_r1.zip,183.519104,https://bulkdata.uspto.gov/data/patent/applica...
4,2023-01-19,ipa230119.zip,183.519079,https://bulkdata.uspto.gov/data/patent/applica...
...,...,...,...,...
72,2023-12-14,ipa231214.zip,132.131911,https://bulkdata.uspto.gov/data/patent/applica...
73,2023-12-21,ipa231221_r1.zip,199.521955,https://bulkdata.uspto.gov/data/patent/applica...
74,2023-12-21,ipa231221.zip,199.523586,https://bulkdata.uspto.gov/data/patent/applica...
75,2023-12-28,ipa231228.zip,197.839358,https://bulkdata.uspto.gov/data/patent/applica...


In [66]:
print(F"Total file size: {round(df["File Size (Mb)"].sum()/1000,2)} GB")

Total file size: 13.38 GB


In [ ]:
import os 
from tqdm import tqdm
import requests

download_path = "patents_data"

if not os.path.exists(download_path):
    os.makedirs(download_path)

for index, row in tqdm(df[:2].iterrows(), total=df[:2].shape[0], desc="Downloading files"):
    file_url = row["Download URL"]
    file_name = row["Date"] + "_" + row["File Name"].split(".")[0]
    zip_file_name = f"{file_name}.zip"
    zip_file_path = os.path.join(download_path, zip_file_name)
    
    response = requests.get(file_url, stream=True)
    with open(zip_file_path, "wb") as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)
    print(f"Downloaded and saved {file_name} as {zip_file_name}")

Downloaded and saved 2023-01-05_ipa230105 as 2023-01-05_ipa230105.zip


Downloaded and saved 2023-01-12_ipa230112_r1 as 2023-01-12_ipa230112_r1.zip


In [76]:
unzip_path = "unzipped_patents_data"

if not os.path.exists(unzip_path):
    os.makedirs(unzip_path)

for file_name in tqdm(os.listdir(download_path), desc="Unzipping files"):
    if file_name.endswith(".zip"):
        zip_file_path = os.path.join(download_path, file_name)
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        print(f"Unzipped {file_name} to {unzip_path}")

Unzipping files:  50%|█████     | 1/2 [00:12<00:12, 12.24s/it]

Unzipped 2023-01-05_ipa230105.zip to unzipped_patents_data


Unzipping files: 100%|██████████| 2/2 [00:17<00:00,  8.84s/it]

Unzipped 2023-01-12_ipa230112_r1.zip to unzipped_patents_data


In [139]:
from lxml import etree
for root, dirs, files in os.walk(unzip_path):
    for file in files:
        if file.endswith(".xml"):
            xml_file_path = os.path.join(root, file)
            try:
                parser = etree.XMLParser(recover=True)
                tree = etree.parse(xml_file_path, parser)
                root_element = tree.getroot()
                # Process the XML data
                print(f"Parsed XML file: {xml_file_path}")
                # Example: print the root tag
                print(f"Root tag: {root_element.tag}")
            except etree.XMLSyntaxError as e:
                print(f"Failed to parse {xml_file_path}: {e}")

Parsed XML file: unzipped_patents_data\ipa230105.xml
Root tag: us-patent-application
Parsed XML file: unzipped_patents_data\ipa230112.xml
Root tag: us-patent-application


In [148]:
for root, dirs, files in os.walk(unzip_path):
    for file in files:
        if file.endswith(".xml"):
            xml_file_path = os.path.join(root, file)
            try:
                parser = etree.XMLParser(recover=True)
                tree = etree.parse(xml_file_path, parser)
                root_element = tree.getroot()
                
                # Example: Extract specific parts using XPath
                # Replace 'your_xpath_expression' with the actual XPath expression
                specific_elements = root_element.xpath('//heading')
                
                for element in specific_elements:
                    print(f"Element tag: {element.tag}, Element text: {element.text}")
                
                print(f"Parsed XML file: {xml_file_path}")
            except etree.XMLSyntaxError as e:
                print(f"Failed to parse {xml_file_path}: {e}")

Element tag: heading, Element text: TECHNICAL FIELD
Element tag: heading, Element text: BACKGROUND
Element tag: heading, Element text: SUMMARY
Element tag: heading, Element text: BRIEF DESCRIPTION OF THE DRAWINGS
Element tag: heading, Element text: REFERENCE NUMBER
Element tag: heading, Element text: DETAILED DESCRIPTION OF THE EMBODIMENTS
Parsed XML file: unzipped_patents_data\ipa230105.xml
Element tag: heading, Element text: CROSS-REFERENCE TO RELATED APPLICATIONS
Element tag: heading, Element text: BACKGROUND
Element tag: heading, Element text: 1. Technical Field
Element tag: heading, Element text: 2. Description of the Background
Element tag: heading, Element text: BRIEF SUMMARY
Element tag: heading, Element text: BRIEF DESCRIPTION OF DRAWINGS
Element tag: heading, Element text: DETAILED DESCRIPTION
Element tag: heading, Element text: Band Saw
Element tag: heading, Element text: Use of Work Tool
Element tag: heading, Element text: Other Embodiments
Element tag: heading, Element tex

In [137]:
from lxml import etree

def extract_text_with_path(element, current_path=""):
    results = []
    
    # Get this element's path
    path = f"{current_path}/{element.tag}"
    
    # Get direct text from this element
    if element.text and element.text.strip():
        results.append((path, element.text.strip()))
    
    # Get text from children
    for child in element:
        results.extend(extract_text_with_path(child, path))
        # Get tail text if any
        if child.tail and child.tail.strip():
            results.append((path, child.tail.strip()))
            
    return results

for root, dirs, files in os.walk(unzip_path):
    for file in files:
        if file.endswith(".xml"):
            xml_file_path = os.path.join(root, file)
            try:
                parser = etree.XMLParser(recover=True)
                tree = etree.parse(xml_file_path, parser)
                root_element = tree.getroot()
                
                # Extract all text with paths
                text_contents = extract_text_with_path(root_element)
                
                print(f"\nProcessing: {xml_file_path}")
                print(f"Found {len(text_contents)} text elements")
                for path, text in text_contents:
                    print(f"\nPath: {path}")
                    print(f"Text: {text[:100]}...")  # First 100 chars
                
            except etree.XMLSyntaxError as e:
                print(f"Failed to parse {xml_file_path}: {e}")


Processing: unzipped_patents_data\ipa230105.xml
Found 600 text elements

Path: /us-patent-application/us-bibliographic-data-application/publication-reference/document-id/country
Text: US...

Path: /us-patent-application/us-bibliographic-data-application/publication-reference/document-id/doc-number
Text: 20230000001...

Path: /us-patent-application/us-bibliographic-data-application/publication-reference/document-id/kind
Text: A1...

Path: /us-patent-application/us-bibliographic-data-application/publication-reference/document-id/date
Text: 20230105...

Path: /us-patent-application/us-bibliographic-data-application/application-reference/document-id/country
Text: US...

Path: /us-patent-application/us-bibliographic-data-application/application-reference/document-id/doc-number
Text: 17364781...

Path: /us-patent-application/us-bibliographic-data-application/application-reference/document-id/date
Text: 20210630...

Path: /us-patent-application/us-bibliographic-data-application/us-applicatio

In [138]:
text_contents

[('/us-patent-application/us-bibliographic-data-application/publication-reference/document-id/country',
  'US'),
 ('/us-patent-application/us-bibliographic-data-application/publication-reference/document-id/doc-number',
  '20230007824'),
 ('/us-patent-application/us-bibliographic-data-application/publication-reference/document-id/kind',
  'A1'),
 ('/us-patent-application/us-bibliographic-data-application/publication-reference/document-id/date',
  '20230112'),
 ('/us-patent-application/us-bibliographic-data-application/application-reference/document-id/country',
  'US'),
 ('/us-patent-application/us-bibliographic-data-application/application-reference/document-id/doc-number',
  '17825930'),
 ('/us-patent-application/us-bibliographic-data-application/application-reference/document-id/date',
  '20220526'),
 ('/us-patent-application/us-bibliographic-data-application/us-application-series-code',
  '17'),
 ('/us-patent-application/us-bibliographic-data-application/priority-claims/priority-cl

In [135]:
import json
from lxml import etree

def xml_to_dict(element):
    result = {}
    
    # Add attributes
    if element.attrib:
        result.update(element.attrib)
    
    # Add children
    for child in element:
        child_dict = xml_to_dict(child)
        if child.tag in result:
            if not isinstance(result[child.tag], list):
                result[child.tag] = [result[child.tag]]
            result[child.tag].append(child_dict)
        else:
            result[child.tag] = child_dict
    
    # Add text content
    if element.text and element.text.strip():
        if result:
            result['#text'] = element.text.strip()
        else:
            result = element.text.strip()
            
    return result

# Parse XML and convert to JSON
parser = etree.XMLParser(recover=True)
tree = etree.parse('unzipped_patents_data\\ipa230112.xml', parser)
root = tree.getroot()

# Convert to dictionary then JSON
xml_dict = xml_to_dict(root)
json_data = json.dumps(xml_dict, indent=2)

# Save or print JSON
with open('output.json', 'w') as f:
    f.write(json_data)
print(json_data)

TypeError: keys must be str, int, float, bool or None, not cython_function_or_method

In [136]:
xml_dict

{'lang': 'EN',
 'dtd-version': 'v4.6 2022-02-17',
 'file': 'US20230007824A1-20230112.XML',
 'status': 'PRODUCTION',
 'id': 'us-patent-application',
 'country': 'US',
 'date-produced': '20221223',
 'date-publ': '20230112',
 'us-bibliographic-data-application': {'lang': 'EN',
  'country': 'US',
  'publication-reference': {'document-id': {'country': 'US',
    'doc-number': '20230007824',
    'kind': 'A1',
    'date': '20230112'}},
  'application-reference': {'appl-type': 'utility',
   'document-id': {'country': 'US',
    'doc-number': '17825930',
    'date': '20220526'}},
  'us-application-series-code': '17',
  'priority-claims': {'priority-claim': {'sequence': '01',
    'kind': 'national',
    'country': 'JP',
    'doc-number': '2021-113046',
    'date': '20210707'}},
  'classifications-ipcr': {'classification-ipcr': {'ipc-version-indicator': {'date': '20060101'},
    'classification-level': 'A',
    'section': 'B',
    'class': '23',
    'subclass': 'D',
    'main-group': '53',
    'sub